<a href = "https://colab.research.google.com/github/flyingzaptop/python-colab/blob/main/%D0%94%D0%BE%D0%B C%D0%D1%88%D0%BD%D1%8f%20%D0%BD%D0%D0%D0%D1%80%D0%D0%D0%BC%D0%D1%80%D0%D0%D0%B6%D1%96.IPYNB "Target =" _ Pare „https://colab.research.google.com/assets/colab-badge.svg” alt = "Otwórz w colab"/> </a>

Import modułów

In [1]:
import torch
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

Dane ubezpieczenia zdrowotnego

In [2]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/insurance.csv')

df['sex'] = df['sex'].map({'male': 0, 'female':1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes':1})

df = df.drop(columns=['region'])

data = df[['age', 'bmi', 'expenses']]
df[['age', 'bmi', 'expenses']] = (data - data.min()) / (data.max() - data.min())

In [3]:
df.head()

,age,sex,bmi,children,smoker,expenses
0,0.021739,1,0.320755,0,1,0.251611
1,0.000000,0,0.479784,1,0,0.009636
2,0.217391,0,0.458221,3,0,0.053115
3,0.326087,0,0.180593,0,0,0.333010
4,0.304348,0,0.347709,0,0,0.043816


Konieczne jest przewidzieć wielkość jego ubezpieczenia „wydatki” na podstawie informacji ludzkich

# Zadanie 1
Utwórz klasę danych

In [5]:
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, X, y):
      self.X = torch.FloatTensor(X.values).to(device)
      self.y = torch.FloatTensor(y.values).to(device)
      self.y = self.y.reshape(-1, 1)

    def __len__(self):
      return len(self.X)

    def __getitem__(self, idx):
      return self.X[idx], self.y[idx]

# Create dataset object
X = df.drop(columns=['expenses'])
y = df['expenses']

dataset = MyDataset(X, y)

## weryfikacja

In [6]:

def check_initialization(X, y):
    try:
        dataset = MyDataset(X, y)
        if torch.is_tensor(dataset.X) and torch.is_tensor(dataset.y):
            print("[OK] Ініціалізація коректна.")
        else:
            print("[FAILED] Ініціалізація не вдалася: X або y не є тензорами.")
    except Exception as e:
        print(f"[FAILED] Помилка ініціалізації: {e}")

def check_y_shape(dataset):
    if dataset.y.ndim == 2 and dataset.y.shape[1] == 1:
        print("[OK] 'y' правильно перетворений на вектор-стовпець.")
    else:
        print(f"[FAILED] 'y' перетворений некоректно, поточна форма: {dataset.y.shape}")

def check_length(dataset, expected_length):
    if len(dataset) == expected_length:
        print("[OK] Перевірка довжини пройдена.")
    else:
        print(f"[FAILED] Перевірка довжини не вдалася. Очікувана довжина: {expected_length}, отримана: {len(dataset)}.")

def check_getitem(dataset, idx):
    try:
        X_item, y_item = dataset[idx]
        if torch.is_tensor(X_item) and torch.is_tensor(y_item):
            print(f"[OK] Отримання елемента коректне для індексу {idx}.")
        else:
            print(f"[FAILED] Отримання елемента некоректне для індексу {idx}.")
    except Exception as e:
        print(f"[FAILED] Помилка у методі `__getitem__`: {e}")

def check_tensor_types(dataset):
    if dataset.X.dtype == torch.float32 and dataset.y.dtype == torch.float32:
        print("[OK] X та y збережені як FloatTensor.")
    else:
        print(f"[FAILED] Невідповідність типу тензорів: X має тип {dataset.X.dtype}, y має тип {dataset.y.dtype}.")

In [7]:
def main_checks():
    print("\n=== Виконання перевірок ===")
    check_initialization(X, y)
    check_y_shape(dataset)
    check_length(dataset, len(X))
    check_getitem(dataset, 0)
    check_tensor_types(dataset)
    print("=== Перевірки завершені ===\n")


main_checks()


=== Виконання перевірок ===
[OK] Ініціалізація коректна.
[OK] 'y' правильно перетворений на вектор-стовпець.
[OK] Перевірка довжини пройдена.
[OK] Отримання елемента коректне для індексу 0.
[OK] X та y збережені як FloatTensor.
=== Перевірки завершені ===



# Zadanie 2
Utwórz DataSaloader. Sprawdź wymiary danych w pierwszym pakiecie. Rozmiar pakietu Wybierz siebie

In [8]:
from torch.utils.data import DataLoader

batch_size = 64
dataloader = DataLoader(dataset,
                        batch_size=batch_size,
                        shuffle=True,
                        generator=torch.Generator(device=device))

## weryfikacja

In [9]:

def check_batch_size(batch_size):
    if isinstance(batch_size, int) and batch_size > 0:
        print(f"[OK] batch_size встановлений як {batch_size}.")
    else:
        print(f"[FAILED] batch_size некоректний, поточне значення: {batch_size}")

def check_dataloader_initialization(dataset, batch_size):
    try:
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        if isinstance(dataloader, DataLoader):
            print(f"[OK] DataLoader створений коректно з batch_size = {batch_size}.")
        else:
            print("[FAILED] DataLoader не створений.")
    except Exception as e:
        print(f"[FAILED] Помилка під час ініціалізації DataLoader: {e}")

def check_dataloader_loading(dataloader, batch_size):
    try:
        for batch in dataloader:
            print(f"[OK] Форма ознак у батчі: {batch[0].shape}, Форма цільових змінних: {batch[1].shape}")
            if batch[0].shape[0] == batch_size or len(dataloader.dataset) % batch_size != 0:
                print(f"[OK] DataLoader завантажує дані пакетами по {batch_size}.")
            else:
                print(f"[FAILED] Розмір батчу некоректний, очікувано: {batch_size}")
            break
    except Exception as e:
        print(f"[FAILED] Помилка під час завантаження DataLoader: {e}")

# Main function to run all checks
def main_checks_dataloader(X, y, batch_size):
    # Create an instance of the dataset
    dataset = MyDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    print("\n=== Виконання перевірок DataLoader ===")
    check_batch_size(batch_size)
    check_dataloader_initialization(dataset, batch_size)
    check_dataloader_loading(dataloader, batch_size)
    print("=== Перевірки завершені ===\n")

In [10]:
def main_checks_dataloader():
    print("\n=== Виконання перевірок DataLoader ===")
    check_batch_size(batch_size)
    check_dataloader_initialization(dataset, batch_size)
    check_dataloader_loading(dataloader, batch_size)
    print("=== Перевірки завершені ===\n")

main_checks_dataloader()


=== Виконання перевірок DataLoader ===
[OK] batch_size встановлений як 64.
[OK] DataLoader створений коректно з batch_size = 64.
[OK] Форма ознак у батчі: torch.Size([64, 5]), Форма цільових змінних: torch.Size([64, 1])
[OK] DataLoader завантажує дані пакетами по 64.
=== Перевірки завершені ===



# Zadanie 3
Utwórz sieć neuronową:
* Wybierz liczbę warstw między 1-5
* Wybierz różne funkcje aktywacji: RELU, Leakyrelu, Swish
*Dzieciak neuronów w ukrytych warstwach powinien ** nie rosnąć **

In [12]:
from torch import nn


model = nn.Sequential(
    nn.Linear(5, 5),
    nn.ReLU(),
    nn.Linear(5, 1)
)

model

Sequential(
  (0): Linear(in_features=5, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=1, bias=True)
)

## weryfikacja

In [13]:
def check_layers(model):
    # Перевірка кількості шарів
    num_layers = len(list(model.children()))
    if 1 <= num_layers <= 5:
        print("[OK] Кількість шарів: ", num_layers)
    else:
        print("[FAILED] Кількість шарів: ", num_layers)

def check_activation_functions(model):
    # Перевірка наявності правильних функцій активації
    valid_activations = [nn.ReLU, nn.LeakyReLU, nn.SiLU]  # SiLU is еквівалент Swish
    layers = list(model.children())

    # Перевірка першого шару
    first_layer = layers[0]
    if isinstance(first_layer, nn.Linear) and first_layer.in_features == dataset.X.shape[1]:
        print(f"[OK] Кількість вхідних нейронів першого шару відповідає {dataset.X.shape[1]}")
    else:
        print("[FAILED] Кількість вхідних нейронів першого шару: ", first_layer.in_features,
              " (очікується: ", dataset.X.shape[1], ")")

    for layer in layers[1:]:  # Починаємо з другого шару
        if isinstance(layer, nn.Linear):
            if layer.in_features < layer.out_features:
                print("[FAILED] Кількість нейронів зростає: ", layer.in_features, "->", layer.out_features)
                return

    print("[OK] Всі функції активації вірні")

In [14]:
def main_check():
    print("Розпочинаю перевірку моделі...")
    check_layers(model)
    check_activation_functions(model)

main_check()

Розпочинаю перевірку моделі...
[OK] Кількість шарів:  3
[OK] Кількість вхідних нейронів першого шару відповідає 5
[OK] Всі функції активації вірні


# Zadanie 4
Określ funkcję straty i optymalizator

In [15]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Zadanie 5
Zawiesić sieć neuronową. Odwiedź niewielką liczbę epok

In [16]:
import numpy as np

num_epochs = 15
losses = []

for epoch in range(num_epochs):
  losses_ = []
  for X_batch, y_batch in dataloader:
        # пряме пошерення
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        losses_.append(loss.item())

        # Зворотнє поширення
        loss.backward() # рахуємо градієнти
        optimizer.step()
        optimizer.zero_grad()

  losses.append(np.mean(losses_))

  if epoch % 1 == 0:
          print(f'loss={losses[-1]}')

loss=0.43520007530848187
loss=0.3741475953942254
loss=0.3240037191481817
loss=0.28293411291780923
loss=0.2482835856221971
loss=0.2194983469588416
loss=0.1947545451777322
loss=0.1740081530241739
loss=0.15599284924211956
loss=0.14060602798348382
loss=0.12749162990422475
loss=0.1158609145453998
loss=0.10614611243917829
loss=0.09742289746091479
loss=0.09005146934872582


# Zadanie 6
Zapisz sieć neuronową i dokonaj prognozy

In [17]:
torch.save(model, 'model.pt')
load_model = torch.load('model.pt')

<ipython-input-17-f2aed6997a05>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_model = torch.load('model.pt')


In [19]:
load_model

Sequential(
  (0): Linear(in_features=5, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=1, bias=True)
)

In [18]:
X_data = torch.FloatTensor(X.iloc[[0]].values)
X_data = X_data.to(device)

print(X_data)
print(X_data.shape)

tensor([[0.0217, 1.0000, 0.3208, 0.0000, 1.0000]])
torch.Size([1, 5])


In [20]:
load_model(X_data)

tensor([[0.2115]], grad_fn=<AddmmBackward0>)